# Runing Mistral-7b AI on a Single GPU with Google Colab
Welcome to this notebook that will show you how to load and run Mistral-7b with QLoRA which is a 4bit quantization technique with no performance degradation.

In this notebook, we will learn together how to load a model in 4bit, understand all its variants and how to run them for inference.

Note that this could be used for any model that supports device_map (i.e. loading the model with accelerate).

## Step 0 -  Enable text wrapping so we don't have to scrool horizontally


In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)


## Step 1 - Install necessary packages
First, install the dependencies below to get started. As these features are available on the main branches only, we need to install the libraries below from source.

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [ ]:
!pip install trl accelerate torch bitsandbytes peft datasets


## Step 2 - Define quantization parameters through the BitsandBytesConfig from transformers


* load_in_4bit=True: specify that we want to convert and load the model in 4-bit precision.
* bnb_4bit_use_double_quant=True: Use nested quantization for more memory efficient inference and training.
* bnd_4bit_quant_type="nf4": The 4bit integration comes with 2 different quantization types FP4 and NF4. The NF4 dtype stands for Normal Float 4 and is introduced in the QLoRA paper. By default, the FP4 quantization is used.
* bnb_4bit_compute_dype=torch.bfloat16: The compute dtype is used to change the dtype that will be used during computation. By default, the compute dtype is set to float32 but computation can be set to bf16 for speedups.



In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



## Step 3 - Load the Model with quantization

Now we specify the model ID and then we load it with our previously defined quantization configuration.

In [5]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", use_cache=False)
tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

If you print the model, you will see that most of the nn.Linear layers are replaced by bnb.nn.Linear4bit layers!

In [7]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )

Let's make sure we loaded the whole model on GPU

In [7]:
model.hf_device_map

{'': 0}

## Step 5 - Once loaded, run a generation!

Test 1: ask a recipe on mayonnaise

In [7]:
device = "cuda:0"

messages = [
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]


encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)


generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Do you have mayonnaise recipes? [/INST] Yes, I have several recipes for mayonnaise. Here are three different versions:

1. Classic Mayonnaise Recipe:
Ingredients:
- 2 egg yolks
- 1 tablespoon Dijon mustard
- 3 tablespoons water
- 1/4 teaspoon salt
- 1 tablespoon white wine vinegar
- 1/2 cup vegetable oil or canola oil, at room temperature

Instructions:

a. In a medium-sized mixing bowl, whisk together the egg yolks, dijon mustard, water, and salt until well combined.

b. Slowly add the white wine vinegar, whisking continuously after each addition.

c. Once the mixture is emulsified and thickened, gradually add the oil, whisking constantly to keep it smooth.

d. Continue whisking for a few minutes until the mixture thickens and reaches your desired consistency.

2. Vegan Mayonnaise Recipe:
Ingredients:
- 1/2 cup canola oil, at room temperature
- 1/4 cup unsweetened applesauce
- 1/4 cup lemon juice
- 2 tablespoons nutritional yeast
- 1 clove garlic
- 1/2 tablespoon Dijon must

Test 2: ask the model to generate python code from natural language instruction

In [ ]:
messages = [
    {"role": "user", "content": "write a python function to generate a list of random 1000 numbers between 1 and 10000?"}
    ]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] write a python function to generate a list of random 1000 numbers between 1 and 10000? [/INST] Sure, here is a simple Python function that generates a list of 1000 random numbers between 1 and 10000:
```
import random

def generate_random_list():
    # generate an empty list
    random_list = []

    # generate 1000 random numbers between 1 and 10000
    for i in range(1, 10001):
        # append the random number to the list
        random_list.append(random.randint(1, 10000))

    # return the list
    return random_list

# call the function
print(generate_random_list())
```
The `random.randint` function is used to generate a random integer between the two arguments provided. In this case, it is used to generate a random number between 1 and 10000, and the `range` function is used to loop over the numbers from 1 to 10000 to generate a list of 1000 random numbers.</s>


Test 3 : ask the model to explain in simple words what is a Large Language Model

In [17]:
device = "cuda:0"


PROMPT= """ ### Instruction: Only generate Answer for Input starting with: "Generate Cypher Query" and generate neo4j Cypher query for the given Input. For any other Input - Answer: "Please ask Cypher query Only"
### Input:
give me recipe for omlet

### Answer:
"""

encodeds = tokenizer(PROMPT, return_tensors="pt", add_special_tokens=True)
model_inputs = encodeds.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])



<s>  ### Instruction: Only generate Answer for Input starting with: "Generate Cypher Query" and generate neo4j Cypher query for the given Input. For any other Input - Answer: "Please ask Cypher query Only"
### Input:
give me recipe for omlet

### Answer:
Please ask Cypher query only.</s>


In [ ]:
device = "cuda:0"
PROMPT= """
### Question:
我想去中国旅游。 你推荐参观哪些地方

### Answer:
"""

encodeds = tokenizer(PROMPT, return_tensors="pt", add_special_tokens=True)
model_inputs = encodeds.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s>  
### Question:
我想去中国旅游。 你推荐参观哪些地方

### Answer:
中国是一个非常大国家，有许多非常漂亮的地方，因此我非常高兴能对你推荐一些参观方案。首先，你可以去中国的首都北京，看看中国历史的首要地标，像皇帝宫殿、辛阳寺、和祥楼之类的。

第二，这里也有许多其他的好地方，例如长城、西天门等。这里我们还可以参观一些非常著名的国家地标，如太阳山、大理城市、香港、鸡城、清原草原等.

第三，如果你喜欢 морpherous 地方，我们还可以去参观中国的海洋地marks，像海洋大象之地、海洋美景之地、海洋世界之地这些地方都非常漂亮。如果你喜欢中国的草原地带，我们可以参观青海草原、青海杜克、青海新革、青海海西、青海海东等地位。

最后，我还想推荐一个非常美景之地--太阳山，它是中国历史、文化和自然的一个非常重要的地标。这里有许多文化的风格，例如古老的住宅、古老的寺庙、古老的草坪，以及许多自然的风格，如美景的湖泊、美景的森林、美景的山区和美景的水falls。

总的来说，去中国旅游有许许多漂亮的地方。我希望我能为您提供一个帮助好的建议，以便您能根据自己的喜好和喜好来参观中国的好地方。</s>


## Creating Dummy Dataset for Cypher

In [8]:
schema = [
  {
    "nodeLabel": "Territory"
  },
  {
    "nodeLabel": "Region"
  },
  {
    "nodeLabel": "Supplier"
  },
  {
    "nodeLabel": "Shipper"
  },
  {
    "nodeLabel": "Product"
  },
  {
    "nodeLabel": "Category"
  },
  {
    "nodeLabel": "Order"
  },
  {
    "nodeLabel": "Customer"
  },
  {
    "nodeLabel": "Employee"
  }
  ,
  {
    "relationshipName": "IN_REGION"
  },
  {
    "relationshipName": "SUPPLIES"
  },
  {
    "relationshipName": "PART_OF"
  },
  {
    "relationshipName": "SHIPS"
  },
  {
    "relationshipName": "PURCHASED"
  },
  {
    "relationshipName": "SOLD"
  },
  {
    "relationshipName": "REPORTS_TO"
  },
  {
    "relationshipName": "IN_TERRITORY"
  },
  {
    "relationshipName": "ORDERS"
  }
]


In [9]:
from datasets import Dataset

# Define the generator
def gen():
    # List of example nodes and relations for the dataset
    examples = [
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Which territory does a specific employee belong to?",
        "Schema": schema,
        "Output": "MATCH (e:Employee)-[:IN_TERRITORY]->(t:Territory) RETURN t.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "List all suppliers in a specific region.",
        "Schema": schema,
        "Output": "MATCH (s:Supplier)-[:IN_REGION]->(r:Region) RETURN s.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Which products are supplied by a specific supplier?",
        "Schema": schema,
        "Output": "MATCH (p:Product)<-[:SUPPLIES]-(s:Supplier) RETURN p.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Who reports to a specific employee?",
        "Schema": schema,
        "Output": "MATCH (e1:Employee)-[:REPORTS_TO]->(e2:Employee) RETURN e1.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Which shippers are involved in a specific order?",
        "Schema": schema,
        "Output": "MATCH (s:Shipper)-[:SHIPS]->(o:Order) RETURN s.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Find all products in a specific category.",
        "Schema": schema,
        "Output": "MATCH (p:Product)-[:PART_OF]->(c:Category) RETURN p.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Which employees are in a specific territory?",
        "Schema": schema,
        "Output": "MATCH (e:Employee)-[:IN_TERRITORY]->(t:Territory) RETURN e.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "List all orders made by a specific customer.",
        "Schema": schema,
        "Output": "MATCH (c:Customer)-[:ORDERS]->(o:Order) RETURN o.id"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Which products are purchased in a specific order?",
        "Schema": schema,
        "Output": "MATCH (p:Product)<-[:PURCHASED]-(o:Order) RETURN p.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "List all suppliers supplying a specific product.",
        "Schema": schema,
        "Output": "MATCH (s:Supplier)-[:SUPPLIES]->(p:Product) RETURN s.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Which regions have a specific territory?",
        "Schema": schema,
        "Output": "MATCH (r:Region)<-[:IN_REGION]-(t:Territory) RETURN r.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Find all employees who sold a specific product.",
        "Schema": schema,
        "Output": "MATCH (e:Employee)-[:SOLD]->(p:Product) RETURN e.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Which products are sold by a specific employee?",
        "Schema": schema,
        "Output": "MATCH (e:Employee)-[:SOLD]->(p:Product) RETURN p.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Which territories fall under a specific region?",
        "Schema": schema,
        "Output": "MATCH (t:Territory)-[:IN_REGION]->(r:Region) RETURN t.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Find the products shipped by a specific shipper.",
        "Schema": schema,
        "Output": "MATCH (s:Shipper)<-[:SHIPS]-(p:Product) RETURN p.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Which customers ordered products from a specific supplier?",
        "Schema": schema,
        "Output": "MATCH (c:Customer)-[:ORDERS]->(:Product)<-[:SUPPLIES]-(s:Supplier) RETURN c.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "List all regions with their associated territories.",
        "Schema": schema,
        "Output": "MATCH (r:Region)<-[:IN_REGION]-(t:Territory) RETURN r.name, collect(t.name)"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Which employees do not report to anyone?",
        "Schema": schema,
        "Output": "MATCH (e:Employee) WHERE NOT (e)-[:REPORTS_TO]->() RETURN e.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Find all customers who ordered a specific product.",
        "Schema": schema,
        "Output": "MATCH (c:Customer)-[:ORDERS]->(p:Product) WHERE p.name = 'SpecificProductName' RETURN c.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Which employees are part of a given territory?",
        "Schema": schema,
        "Output": "MATCH (e:Employee)-[:IN_TERRITORY]->(t:Territory) RETURN e.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Find the most common product supplied by suppliers.",
        "Schema": schema,
        "Output": "MATCH (s:Supplier)-[:SUPPLIES]->(p:Product) RETURN p.name, count(s) AS suppliers_count ORDER BY suppliers_count DESC LIMIT 1"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "Which products are not supplied by any supplier?",
        "Schema": schema,
        "Output": "MATCH (p:Product) WHERE NOT (p)<-[:SUPPLIES]-(:Supplier) RETURN p.name"
    },
    {
        "Instruction": "Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only",
        "Input": "List all shippers who have shipped more than 10 products.",
        "Schema": schema,
        "Output": "MATCH (s:Shipper)-[:SHIPS]->(p:Product) WITH s, count(p) AS product_count WHERE product_count > 10 RETURN s.name"
    }


    ]




    for example in examples:
        yield example

# Generate the dataset
cypher_dataset = Dataset.from_generator(gen)


Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
# creating formating fucntion

def create_prompt_instruction(sample):
   return f"""### Instruction:
   Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only

   ### Input
   {sample['Input']}

   ### Schema
   {sample['Schema']}

   ### Response:
   {sample['Output']}
   """


Dataset({
    features: ['Instruction', 'Input', 'Schema', 'Output'],
    num_rows: 23
})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [23]:
print(create_prompt_instruction(cypher_dataset[0]))


### Instruction: 
   Generate Cypher queries to query a Neo4j graph database based on the provided schema definition. All other questions can be replied with - please ask Cypher related questions only 

   ### Input 
   Which territory does a specific employee belong to?

   ### Schema
   [{'nodeLabel': 'Territory', 'relationshipName': None}, {'nodeLabel': 'Region', 'relationshipName': None}, {'nodeLabel': 'Supplier', 'relationshipName': None}, {'nodeLabel': 'Shipper', 'relationshipName': None}, {'nodeLabel': 'Product', 'relationshipName': None}, {'nodeLabel': 'Category', 'relationshipName': None}, {'nodeLabel': 'Order', 'relationshipName': None}, {'nodeLabel': 'Customer', 'relationshipName': None}, {'nodeLabel': 'Employee', 'relationshipName': None}, {'nodeLabel': None, 'relationshipName': 'IN_REGION'}, {'nodeLabel': None, 'relationshipName': 'SUPPLIES'}, {'nodeLabel': None, 'relationshipName': 'PART_OF'}, {'nodeLabel': None, 'relationshipName': 'SHIPS'}, {'nodeLabel': None, 'relation

In [10]:
cypher_dataset = cypher_dataset.train_test_split(test_size=0.1)

train_dataset = cypher_dataset['train']


## Fine Tuning Mistral

In [11]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

# PEFT Config
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)


# Prepare the model for finetuning
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [14]:
# Define training arguments
args = TrainingArguments(
    output_dir = "mistral_instruct_qa",
    num_train_epochs = 1,
    per_device_train_batch_size = 4,
    warmup_steps = 0.03,
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay = 0.001,
    max_grad_norm = 0.3,
    optim = "paged_adamw_32bit",
    bf16=False,
    fp16=False,
    lr_scheduler_type='cosine',
    disable_tqdm=True
)


# Define SFTTrainer arguments
max_seq_length = 1000

trainer = SFTTrainer(
    model=model,
    peft_config=peft_config,
   max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=create_prompt_instruction,
    args=args,
    train_dataset=train_dataset,
)

In [16]:
# kick off the finetuning job
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'train_runtime': 95.587, 'train_samples_per_second': 0.209, 'train_steps_per_second': 0.052, 'train_loss': 0.5671390295028687, 'epoch': 0.4}


TrainOutput(global_step=2, training_loss=0.5671390295028687, metrics={'train_runtime': 95.587, 'train_samples_per_second': 0.209, 'train_steps_per_second': 0.052, 'train_loss': 0.5671390295028687, 'epoch': 0.4})

In [19]:
trainer.save_model("mistral_instruct_cypher")

In [6]:
# Load the finetuned model
finetuned_model = AutoPeftModelForCausalLM.from_pretrained(
    "mistral_instruct_cypher",
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
  device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistral_instruct_cypher")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
device = "cuda:0"


PROMPT= """ ### Instruction: Generate Cypher queries to query a Neo4j graph database based on the provided schema. All other questions can be replied with - please ask Cypher related questions only
        ### Input: Find all customers who ordered a specific product,
        ### Schema:[{'nodeLabel': 'Territory', 'relationshipName': None}, {'nodeLabel': 'Region', 'relationshipName': None}, {'nodeLabel': 'Supplier', 'relationshipName': None}, {'nodeLabel': 'Shipper', 'relationshipName': None}, {'nodeLabel': 'Product', 'relationshipName': None}, {'nodeLabel': 'Category', 'relationshipName': None}, {'nodeLabel': 'Order', 'relationshipName': None}, {'nodeLabel': 'Customer', 'relationshipName': None}, {'nodeLabel': 'Employee', 'relationshipName': None}, {'nodeLabel': None, 'relationshipName': 'IN_REGION'}, {'nodeLabel': None, 'relationshipName': 'SUPPLIES'}, {'nodeLabel': None, 'relationshipName': 'PART_OF'}, {'nodeLabel': None, 'relationshipName': 'SHIPS'}, {'nodeLabel': None, 'relationshipName': 'PURCHASED'}, {'nodeLabel': None, 'relationshipName': 'SOLD'}, {'nodeLabel': None, 'relationshipName': 'REPORTS_TO'}, {'nodeLabel': None, 'relationshipName': 'IN_TERRITORY'}, {'nodeLabel': None, 'relationshipName': 'ORDERS'}],
        ### Output :
"""

encodeds = tokenizer(PROMPT, return_tensors="pt", add_special_tokens=True)
model_inputs = encodeds.to(device)

generated_ids = finetuned_model.generate(**model_inputs, max_new_tokens=500, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s>  ### Instruction: Generate Cypher queries to query a Neo4j graph database based on the provided schema. All other questions can be replied with - please ask Cypher related questions only
        ### Input: Find all customers who ordered a specific product,
        ### Schema:[{'nodeLabel': 'Territory', 'relationshipName': None}, {'nodeLabel': 'Region', 'relationshipName': None}, {'nodeLabel': 'Supplier', 'relationshipName': None}, {'nodeLabel': 'Shipper', 'relationshipName': None}, {'nodeLabel': 'Product', 'relationshipName': None}, {'nodeLabel': 'Category', 'relationshipName': None}, {'nodeLabel': 'Order', 'relationshipName': None}, {'nodeLabel': 'Customer', 'relationshipName': None}, {'nodeLabel': 'Employee', 'relationshipName': None}, {'nodeLabel': None, 'relationshipName': 'IN_REGION'}, {'nodeLabel': None, 'relationshipName': 'SUPPLIES'}, {'nodeLabel': None, 'relationshipName': 'PART_OF'}, {'nodeLabel': None, 'relationshipName': 'SHIPS'}, {'nodeLabel': None, 'relationshipName':

## Saving the Model

In [10]:
!huggingface-cli login
new_model="mistral_instruct_cypher"
finetuned_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

adapter_model.bin:   0%|          | 0.00/218M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sarangsonar/mistral_instruct_cypher/commit/3ea5cc19461a73b12cad5fff61b5fffff6156d43', commit_message='Upload tokenizer', commit_description='', oid='3ea5cc19461a73b12cad5fff61b5fffff6156d43', pr_url=None, pr_revision=None, pr_num=None)

## Loading the Model from hugging face

In [11]:
model_id = "sarangsonar/mistral_instruct_cypher"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", use_cache=False)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
device = "cuda:0"


PROMPT= """ ### Instruction: Generate Cypher queries to query a Neo4j graph database based on the provided schema. All other questions can be replied with - please ask Cypher related questions only
        ### Input: Generate cypher query for - Find all customers who ordered a specific product,
        ### Schema:[{'nodeLabel': 'Territory', 'relationshipName': None}, {'nodeLabel': 'Region', 'relationshipName': None}, {'nodeLabel': 'Supplier', 'relationshipName': None}, {'nodeLabel': 'Shipper', 'relationshipName': None}, {'nodeLabel': 'Product', 'relationshipName': None}, {'nodeLabel': 'Category', 'relationshipName': None}, {'nodeLabel': 'Order', 'relationshipName': None}, {'nodeLabel': 'Customer', 'relationshipName': None}, {'nodeLabel': 'Employee', 'relationshipName': None}, {'nodeLabel': None, 'relationshipName': 'IN_REGION'}, {'nodeLabel': None, 'relationshipName': 'SUPPLIES'}, {'nodeLabel': None, 'relationshipName': 'PART_OF'}, {'nodeLabel': None, 'relationshipName': 'SHIPS'}, {'nodeLabel': None, 'relationshipName': 'PURCHASED'}, {'nodeLabel': None, 'relationshipName': 'SOLD'}, {'nodeLabel': None, 'relationshipName': 'REPORTS_TO'}, {'nodeLabel': None, 'relationshipName': 'IN_TERRITORY'}, {'nodeLabel': None, 'relationshipName': 'ORDERS'}],
        ### Output :
"""

encodeds = tokenizer(PROMPT, return_tensors="pt", add_special_tokens=True)
model_inputs = encodeds.to(device)

generated_ids = finetuned_model.generate(**model_inputs, max_new_tokens=500, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s>  ### Instruction: Generate Cypher queries to query a Neo4j graph database based on the provided schema. All other questions can be replied with - please ask Cypher related questions only
        ### Input: Generate cypher query for - Find all customers who ordered a specific product,
        ### Schema:[{'nodeLabel': 'Territory', 'relationshipName': None}, {'nodeLabel': 'Region', 'relationshipName': None}, {'nodeLabel': 'Supplier', 'relationshipName': None}, {'nodeLabel': 'Shipper', 'relationshipName': None}, {'nodeLabel': 'Product', 'relationshipName': None}, {'nodeLabel': 'Category', 'relationshipName': None}, {'nodeLabel': 'Order', 'relationshipName': None}, {'nodeLabel': 'Customer', 'relationshipName': None}, {'nodeLabel': 'Employee', 'relationshipName': None}, {'nodeLabel': None, 'relationshipName': 'IN_REGION'}, {'nodeLabel': None, 'relationshipName': 'SUPPLIES'}, {'nodeLabel': None, 'relationshipName': 'PART_OF'}, {'nodeLabel': None, 'relationshipName': 'SHIPS'}, {'nodeLabel